# Time-Dependent Routing

## Functions

In [56]:
def get_unique_nodes(edge_list):
    """
    Assembles a list of all unique nodes appearing in a given graph.
    The start node is added by default; every other node is only included
    if it can be reached from another node. Isolated nodes will not be 
    added to the list, unless they happen to be the start node.
    """

    nr_nodes = 0
    unique_nodes = []
    unique_nodes.append(0)

    for edge in edge_list:
        if edge[1] not in unique_nodes:
            unique_nodes.append(edge[1])

    return unique_nodes


In [57]:
def construct_sliced_graph(edge_list, start, dest):
    """
    Constructs a two-dimensional array from a list of edges. Each subarray represents 
    one time slice. The returned array contains the specified start node in the 
    first sub-array; based on this start node, every node that can be reached in 
    n steps is present in the nth subarray. Nodes that can be reached in a different 
    amount of steps are present in every fitting subarray.
    """

    sg = [] # sliced graph
    sg.append([start])
    node_counter = 1


    total_nodes = len(get_unique_nodes(edge_list))

    for k in range(total_nodes):
    
        if len(sg[k]) != 0 and not (len(sg[k]) == 1 and sg[k][0] == dest):
            sg.append([])
            for j in range(len(edge_list)):
                # for a given edge j in the format a->b:
                # if a is in previous slice and b is not in this slice, and if b is not the start node, add it to the slice
                if edge_list[j][0] in sg[k] and edge_list[j][1] not in sg[k+1] and edge_list[j][1] != start:
                    sg[k+1].append(edge_list[j][1])
        else:
            break

    # necessary if graph is not acyclic
    if dest in sg[-1]:
        sg[-1] = [dest]
    else:
        return None 

    # the last node is always the destination node.
    # therefore we can eliminate all nodes in the next-to-last slice that do not lead to the destination node.
    # going back that way, we can eliminate nodes that won't lead to our destination (dead ends).
    
    nr_slices_to_remove = 0

    for i in range(len(sg) - 1):
        nodes_to_remove = []
        index = len(sg) - 1 - i
        for j in range(len(sg[index - 1])):
            leads_to_next_slice = 0 # number of edges in the next slice that sg[index-1][j] is connected to
            for k in range(len(sg[index])):

                # for each node in the next slice, add 1 if sg[index-1][j] is connected to it, 0 if not
                leads_to_next_slice += next((1 for u, v in enumerate(edge_list) if v[0] == sg[index-1][j] and v[1] == sg[index][k]), 0) 

            # if leads_to_next_slice is zero, it means sg[index-1][j] is not connected to any node in the next slice; essentially a dead end. 
            # it is removed, and any node in the previous slice that only connected to it will thus also be a dead end, and will be removed. etc., etc.
            if leads_to_next_slice == 0:
                nodes_to_remove.append(sg[index-1][j])

        for node in nodes_to_remove:
            sg[index-1].remove(node)

    slices_to_remove = []
    for i in range(len(sg)):
        if len(sg[len(sg) - 1 - i]) == 1 and dest in sg[len(sg) - 1 - i]:
            slices_to_remove.append(len(sg) - 1 - i)
        else:
            break
    
    if len(slices_to_remove) > 1:
        for i in slices_to_remove[:-1]:
            del sg[i]

    return sg

In [58]:
def construct_numbered_sliced_graph(sg): 
    """
    Based on a two-dimensional array that contains time slices from a graph (sliced graph),
    renumbers the contents by giving each element its index in the corresponding 
    flattened array. The numbered sliced graph is used for getting the qubit 
    indices of the nodes.
    Example: With input ([[0], [1, 3], [2, 3]]), output would be [[0], [1, 2], [3, 4]].
    """
    
    nsg = [] # numbered sliced graph
    node_counter = 0

    for m in range(len(sg)):
        nsg.append([])
        for n in range(len(sg[m])):
            nsg[m].append(node_counter)
            node_counter += 1
    
    return nsg

In [59]:
def construct_graph_matrix(edge_list):
    """
    Constructs a dataframe that contains all edges present in 
    the graph. The returned dataframe can contain several edges 
    per two nodes. The entry at matrix[0][1] contains an array 
    of all edges that go from node 0 to node 1.
    """

    node_labels = get_unique_nodes(edge_list)
    graph_matrix = pd.DataFrame([[[] for _ in range(len(node_labels))] for _ in range(len(node_labels))], node_labels, node_labels)
    
    for edge in edge_list:
        start = edge[0]
        end = edge[1]
        graph_matrix[start][end].append(edge)
    
    return graph_matrix

In [60]:
def construct_cost_matrix(edge_matrix):
    """
    Constructs a dataframe that contains the cost for the edge 
    between any given pair of nodes a and b. If there is no 
    outgoing edge from a to b, the cost will be equal to the
    fixed penalty value. If there is exactly one edge, its cost
    will be used. If there are several, the smallest cost is 
    used.
    """

    node_labels = edge_matrix.columns
    cost_matrix = pd.DataFrame([[[] for _ in range(len(node_labels))] for _ in range(len(node_labels))], node_labels, node_labels)

    for i in node_labels:
        for j in node_labels:
            entries = edge_matrix[i][j]
            if len(entries) == 0:
                cost_matrix[i][j] = (penalty_value)
            elif len(entries) == 1:
                cost_matrix[i][j] = (entries[0][2])
            else:
                best_cost = penalty_value # penalty by default bigger than the biggest edge cost
                for entry in entries:
                    if entry[2] < best_cost:
                        best_cost = entry[2]
                cost_matrix[i][j] = best_cost
    return cost_matrix

In [61]:
def find_index(node, time_slice, sliced_graph):
    """
    For a given node, time slice, and sliced graph, determines the qubit index 
    of the node. If the node is not present in the given slice in the graph, 
    returns -1.
    """
    
    numbered_graph = construct_numbered_sliced_graph(sliced_graph)
    for i in range(len(sliced_graph[time_slice])):
        if sliced_graph[time_slice][i] == node:
            return numbered_graph[time_slice][i]
    else:
        return -1

In [62]:
def fix_nodes(start, destination, sliced_graph, cost_func):
    """
    In the given cost function, fixes specific nodes according to already known 
    values for the qubits. The qubits representing the start and destination nodes 
    are fixed at 1; any other nodes in the last time slice are fixed at 0.
    """
    
    # set start node to 1
    start_node_index = 'X_' + str(find_index(start, 0, sliced_graph))
    cost_func.linear_constraint(linear={start_node_index: 1}, sense='==', rhs=1, name='start_node')

    # set destination node to 1
    dest_node_index = 'X_' + str(find_index(destination, -1, sliced_graph))
    cost_func.linear_constraint(linear={dest_node_index: 1}, sense='==', rhs=1, name='destination_node')

    # no need to set other nodes in the first and last slice to 0 because there are none (see preprocessing).

    return

In [63]:
def get_node_info_from_index(qubit_index, sg, nsg):
    """ For a given qubit index, returns the corresponding time slice and the node label. """

    for time_slice in range(len(nsg)):
        if qubit_index in nsg[time_slice]:

            pos = nsg[time_slice].index(qubit_index)
            node_label = sg[time_slice][pos]

            return time_slice, node_label

In [90]:
def result_evaluation(result, sg, nsg, cost_matrix):
    """
    For a given qubit string, returns an explanation of the nodes that are passed per time slice.
    """
    
    counter = 0
    previous_node = None
    total_cost = 0
    solution_not_valid = False

    for i in range(len(result)):
        if result[i] == '1':
            time_slice, node_label = get_node_info_from_index(i, sg, nsg)
            if time_slice != counter:
                solution_not_valid = True

            if counter == 0:
                previous_node = node_label
            else:
                cost = cost_matrix[previous_node][node_label]
                total_cost += cost
                print(('\t\t\t\tcost: {}').format(cost))
                previous_node = node_label


            print(('node {} at time slice {}').format(node_label, time_slice))

            counter += 1

            
    print(('\ntotal cost: {}').format(total_cost))
    if total_cost >= penalty_value or solution_not_valid:
        print('\nIt seems like no valid solution could be found. This solution disregards at least one constraint.')

    return

In [65]:
def start_dest_constraints(vso, vti, cost_func):
    """
    Adds constraints to ensure that there is exactly one outgoing edge from the start 
    and one incoming edge to the target vertex.
    """
    
    linear_dict_start = {}
    linear_dict_target = {}

    for i in range(len(vso)):
        linear_dict_start.update({'X_' + str(vso[i]): 1})

    for i in range(len(vti)):
        linear_dict_target.update({'X_' + str(vti[i]): 1})

    cost_func.linear_constraint(linear=linear_dict_start, sense='==', rhs=1, name='start_one_out')
    cost_func.linear_constraint(linear=linear_dict_target, sense='==', rhs=1, name='target_one_inc')

    return

## Preprocessing

In [66]:
import sympy as sym
import matplotlib.pyplot as plt
import pandas as pd

In [67]:
# there are two examples defined in the following, each has a graph as a collection of edges with the format (start, end, edge weight)
# as well as a defined start and destination node
# just comment out the example you don't want to use. if both are left uncommented, the second one will be executed.

# --- first example ---
graph = [(0, 1, 6.0), (0, 2, 4.0), (0, 3, 1.0), (0, 5, 2.0), 
(1, 6, 1.0), 
(2, 4, 3.0), (2, 6, 2.0), (2, 9, 5.0), 
(3, 2, 2.0), (3, 4, 4.0), 
(4, 7, 4.0), (4, 9, 2.0), 
(5, 1, 5.0), 
(6, 8, 3.0), (6, 9, 3.0), 
(7, 9, 3.0),
(9, 8, 4.0)]

start_node = 0
destination_node = 9

# --- second example ---
graph = [(0, 1, 5.0), (0, 2, 4.0), (0, 3, 4.0), 
(1, 4, 3.0), 
(2, 1, 2.0), (2, 3, 1.0), 
(3, 4, 2.0), (3, 5, 3.0), 
(4, 6, 2.0), 
(5, 7, 4.0)]

start_node = 0
destination_node = 4

In [68]:
# the penalty value can be changed

penalty_value = 50

In [69]:
# discard all edges that go out from destination, and all edges that lead to start
graph = [edge for edge in graph if edge[0] != destination_node and edge[1] != start_node] 
graph.append((destination_node, destination_node, 0.0))
graph

[(0, 1, 5.0),
 (0, 2, 4.0),
 (0, 3, 4.0),
 (1, 4, 3.0),
 (2, 1, 2.0),
 (2, 3, 1.0),
 (3, 4, 2.0),
 (3, 5, 3.0),
 (5, 7, 4.0),
 (4, 4, 0.0)]

In [70]:
unique_nodes = get_unique_nodes(graph)
nr_nodes = len(unique_nodes)

In [71]:
# construct sliced graph
test_sg = construct_sliced_graph(graph, start_node, destination_node)

nr_qubits = 0
for i in range(len(test_sg)):
    for j in range(len(test_sg[i])):
        nr_qubits += 1

nr_qubits

8

In [72]:
# print sliced graph
test_sg

[[0], [1, 2, 3], [4, 1, 3], [4]]

In [73]:
# construct and print sliced graph
test_nsg = construct_numbered_sliced_graph(test_sg)
test_nsg

[[0], [1, 2, 3], [4, 5, 6], [7]]

In [74]:
# construct and print graph matrix (shows edges in a matrix layout, except for those edges eliminated in the beginning)
matrix = construct_graph_matrix(graph)
matrix

,0,1,2,3,4,5,7
0,[],[],[],[],[],[],[]
1,"[(0, 1, 5.0)]",[],"[(2, 1, 2.0)]",[],[],[],[]
2,"[(0, 2, 4.0)]",[],[],[],[],[],[]
3,"[(0, 3, 4.0)]",[],"[(2, 3, 1.0)]",[],[],[],[]
4,[],"[(1, 4, 3.0)]",[],"[(3, 4, 2.0)]","[(4, 4, 0.0)]",[],[]
5,[],[],[],"[(3, 5, 3.0)]",[],[],[]
7,[],[],[],[],[],"[(5, 7, 4.0)]",[]


In [75]:
# construct and print edge cost matrix, which only shows the cost of one (in this case, the shortest) edge (or, if no edge exists, the penalty value)
edge_cost_matrix = construct_cost_matrix(matrix)
edge_cost_matrix

,0,1,2,3,4,5,7
0,50,50,50,50,50,50,50
1,5,50,2,50,50,50,50
2,4,50,50,50,50,50,50
3,4,50,1,50,50,50,50
4,50,3,50,2,0,50,50
5,50,50,50,3,50,50,50
7,50,50,50,50,50,4,50


## Cost Function

In [76]:
# define all necessary variables

X = sym.IndexedBase('X') 
c = sym.symbols('c') # counter variable for slices
v = sym.symbols('v') # counter variable for vertices (nodes)
y = sym.symbols('y') # counter variable for vertices
q = sym.symbols('q') # counter variable for slices
w = sym.symbols('w') # counter variable for vertices
P = sym.symbols('P') # penalty value

d = sym.IndexedBase('d') # represents the function d, which returns the edge cost between two vertices

sg = sym.IndexedBase('sg') # sliced graph
nsg = sym.IndexedBase('nsg') # numbered sliced graph

nrslices = sym.symbols('nrslices') # number of slices
lenslice = sym.IndexedBase('lenslice') # length of a given slice

In [77]:
cost_function = sym.Sum(                                # iteration over slices
                    (sym.Sum(                           # iteration over nodes in that slice. in each slice, exactly one node needs to be passed
                        X[nsg[c,v]],                    
                        (v, 0, lenslice[c] - 1)
                        )
                        - 1 )**2 * P,

                    (c, 0, nrslices-1)) + 0.5 * sym.Sum(   
                        2 * (sym.Sum(  sym.Sum(  X[nsg[q,y]] * X[nsg[q+1,w]] * d[sg[q,y], sg[q+1,w]],  (w, 0, lenslice[q + 1] - 1) ) , (y, 0, lenslice[q] - 1))),
                        (q, 0, nrslices - 2))

# workaround -- multiplying with 0.5 and then 2, since sympy does not evaluate the triple sum otherwise
                    
cost_function

Sum(P*(Sum(X[nsg[c, v]], (v, 0, lenslice[c] - 1)) - 1)**2, (c, 0, nrslices - 1)) + 0.5*Sum(2*Sum(X[nsg[q + 1, w]]*X[nsg[q, y]]*d[sg[q, y], sg[q + 1, w]], (w, 0, lenslice[q + 1] - 1), (y, 0, lenslice[q] - 1)), (q, 0, nrslices - 2))

In [78]:
# translation of data into dictionaries for sympy

single_valued_dict = {
    nrslices: len(test_sg),
    P: penalty_value
    }

numbered_sliced_graph_dict = {
    nsg[i, j]: test_nsg[i][j] for i in range(len(test_nsg)) for j in range(len(test_nsg[i]))
}

sliced_graph_dict = {
    sg[i, j]: test_sg[i][j] for i in range(len(test_sg)) for j in range(len(test_sg[i]))
}

len_slice_dict = {
    lenslice[i]: len(test_sg[i]) for i in range(len(test_sg))
}

d_dict = {
    d[i, j]: edge_cost_matrix[i][j] 
    for i in unique_nodes
    for j in unique_nodes
}

# definition of the cost polynomial
cost_poly = sym.Poly(cost_function
                     .subs(single_valued_dict)
                     .doit()
                     .subs(len_slice_dict)
                     .doit()
                     .subs(numbered_sliced_graph_dict)
                     .subs(sliced_graph_dict)
                     .doit()
                     .subs(d_dict)
                     .doit(),
                     [X[i] for i in range(nr_qubits)])
cost_poly

# the only variables in the cost polynomial should be X_0, X_1, X_2 etc. if not, then some of the variables have not been evaluated; order of evaluation is important.
# if the evaluation of parts of the polynomial depends on the evaluation of another, then doit() is necessary in between in order to evaluate that first part.

Poly(50.0*X[0]**2 + 5.0*X[0]*X[1] + 4.0*X[0]*X[2] + 4.0*X[0]*X[3] - 100.0*X[0] + 50.0*X[1]**2 + 100.0*X[1]*X[2] + 100.0*X[1]*X[3] + 3.0*X[1]*X[4] + 50.0*X[1]*X[5] + 50.0*X[1]*X[6] - 100.0*X[1] + 50.0*X[2]**2 + 100.0*X[2]*X[3] + 50.0*X[2]*X[4] + 2.0*X[2]*X[5] + 1.0*X[2]*X[6] - 100.0*X[2] + 50.0*X[3]**2 + 2.0*X[3]*X[4] + 50.0*X[3]*X[5] + 50.0*X[3]*X[6] - 100.0*X[3] + 50.0*X[4]**2 + 100.0*X[4]*X[5] + 100.0*X[4]*X[6] - 100.0*X[4] + 50.0*X[5]**2 + 100.0*X[5]*X[6] + 3.0*X[5]*X[7] - 100.0*X[5] + 50.0*X[6]**2 + 2.0*X[6]*X[7] - 100.0*X[6] + 50.0*X[7]**2 - 100.0*X[7] + 200.0, X[0], X[1], X[2], X[3], X[4], X[5], X[6], X[7], domain='RR')

## Setup

In [79]:
import qiskit
from qiskit.algorithms import QAOA, VQE
import time

from qiskit_optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer, CplexOptimizer, GroverOptimizer
from qiskit.utils import QuantumInstance
from qiskit_optimization.problems import QuadraticProgram
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP

# generate qiskit's cost function
qiskit_cost_function = QuadraticProgram()

# define qiskit variables
for i in range(nr_qubits):
    qiskit_cost_function.binary_var('X_' + str(i))

# specify qiskit cost function
qiskit_cost_function.minimize(
    linear = [int(cost_poly.coeff_monomial(X[i]**1)) for i in range(nr_qubits)],
    quadratic = {
        ('X_'+str(i), 'X_'+str(j)): cost_poly.coeff_monomial(X[i]**1 * X[j]**1)
        for i in range(nr_qubits)
        for j in range(i,nr_qubits)
    }
    )
fix_nodes(start_node, destination_node, test_sg, qiskit_cost_function)

print(qiskit_cost_function.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 100 X_0 - 100 X_1 - 100 X_2 - 100 X_3 - 100 X_4 - 100 X_5 - 100 X_6
      - 100 X_7 + [ 100 X_0^2 + 10 X_0*X_1 + 8 X_0*X_2 + 8 X_0*X_3 + 100 X_1^2
      + 200 X_1*X_2 + 200 X_1*X_3 + 6 X_1*X_4 + 100 X_1*X_5 + 100 X_1*X_6
      + 100 X_2^2 + 200 X_2*X_3 + 100 X_2*X_4 + 4 X_2*X_5 + 2 X_2*X_6
      + 100 X_3^2 + 4 X_3*X_4 + 100 X_3*X_5 + 100 X_3*X_6 + 100 X_4^2
      + 200 X_4*X_5 + 200 X_4*X_6 + 100 X_5^2 + 200 X_5*X_6 + 6 X_5*X_7
      + 100 X_6^2 + 4 X_6*X_7 + 100 X_7^2 ]/2
Subject To
 start_node: X_0 = 1
 destination_node: X_7 = 1

Bounds
 0 <= X_0 <= 1
 0 <= X_1 <= 1
 0 <= X_2 <= 1
 0 <= X_3 <= 1
 0 <= X_4 <= 1
 0 <= X_5 <= 1
 0 <= X_6 <= 1
 0 <= X_7 <= 1

Binaries
 X_0 X_1 X_2 X_3 X_4 X_5 X_6 X_7
End



In [80]:
# this is not included in the cost function that will be passed to the algorithm, so it will be added later to get the actual result
offset = cost_poly.coeff_monomial(1)
offset

200.000000000000

## QAOA

In [81]:
# execute QAOA on local simulator

# https://qiskit.org/documentation/stubs/qiskit.algorithms.QAOA.html?highlight=qaoa

# 'depth of circuit grows linearly with p times (at worst) the number of constraints'
# https://arxiv.org/pdf/1411.4028.pdf
# (p is the parameter reps)

optimizer = SPSA(maxiter=500)

qaoa = QAOA(reps=5,optimizer=optimizer,quantum_instance =
             QuantumInstance(backend=qiskit.Aer.get_backend('qasm_simulator')))
optimizer_qaoa = MinimumEigenOptimizer(qaoa)

results = []

for i in range(10):
    print(i)
    result_qaoa = optimizer_qaoa.solve(qiskit_cost_function)
    results.append(result_qaoa)

0
1
2
3
4
5
6
7
8
9


In [82]:
# actual_opt_cost: optimization cost. includes path cost and incurred penalties. always > 0.
# path: qubit-result
result_df = pd.DataFrame(columns = ['actual_opt_cost', 'path'])

for r in results:
    path_string = str(r.x).replace(' ', '').replace('.', '')[1:-1]
    result_df = result_df.append({'actual_opt_cost': r.fval + offset, 'path': path_string}, ignore_index=True)

print("QAOA:")
print(result_df.sort_values(by=['actual_opt_cost']))

QAOA:
    actual_opt_cost      path
1  6.00000000000000  10011001
2  6.00000000000000  10011001
3  6.00000000000000  10011001
4  6.00000000000000  10011001
5  6.00000000000000  10011001
7  6.00000000000000  10011001
8  6.00000000000000  10011001
9  6.00000000000000  10011001
6  7.00000000000000  10100011
0  8.00000000000000  11001001


## VQE

In [83]:
from qiskit.circuit.library import TwoLocal

backend = qiskit.Aer.get_backend('qasm_simulator')
optimizer = SPSA(maxiter=500)
ry = TwoLocal(nr_qubits, 'ry', 'cz', reps=5, entanglement='linear')
vqe = VQE(ry, optimizer=optimizer, quantum_instance=QuantumInstance(backend=backend))

optimizer_vqe = MinimumEigenOptimizer(vqe)

results_vqe = []

for i in range(10):
    print(i)
    result_vqe = optimizer_vqe.solve(qiskit_cost_function)
    results_vqe.append(result_vqe)

0
1
2
3
4
5
6
7
8
9


In [84]:
# actual_opt_cost: optimization cost. includes path cost and incurred penalties. always > 0.
# path: qubit-result
result_vqe_df = pd.DataFrame(columns = ['actual_opt_cost', 'path'])

for r in results_vqe:
    path_string = str(r.x).replace(' ', '').replace('.', '')[1:-1]
    result_vqe_df = result_vqe_df.append({'actual_opt_cost': r.fval + offset, 'path': path_string}, ignore_index=True)

print("VQE:")
print(result_vqe_df.sort_values(by=['actual_opt_cost']))

VQE:
    actual_opt_cost      path
0  6.00000000000000  10011001
1  6.00000000000000  10011001
2  6.00000000000000  10011001
3  6.00000000000000  10011001
4  6.00000000000000  10011001
5  6.00000000000000  10011001
6  6.00000000000000  10011001
7  6.00000000000000  10011001
9  6.00000000000000  10011001
8  7.00000000000000  10100011


In [87]:
# printing the path for solution 4
# for printing another result, change the first number in the iloc call
result_evaluation(result_vqe_df.iloc[4,1], test_sg, test_nsg, edge_cost_matrix) 

node 0 at time slice 0
				cost: 4.0
node 3 at time slice 1
				cost: 2.0
node 4 at time slice 2
				cost: 0.0
node 4 at time slice 3

total cost: 6.0


## CplexOptimizer

In [88]:
# ------ CplexOptimizer needs to be installed for this and the next cell to work ------

# optimizer = CplexOptimizer() if CplexOptimizer.is_cplex_installed() else None

# results_classic = []

# for i in range(2):
#     result = optimizer.solve(qiskit_cost_function)
#     results_classic.append(result)
# results_classic

In [89]:
### actual_opt_cost: optimization cost. includes path cost and incurred penalties. always > 0.
### path: qubit-result

# result_classic_df = pd.DataFrame(columns = ['optimization_cost', 'path'])
# for r in results_classic:
#     result_string = str(r.x).replace(' ', '').replace('.', '').replace('-', '')[1:-1]
#     result_classic_df = result_classic_df.append({'optimization_cost': r.fval + offset, 'path': result_string}, ignore_index=True)
# result_classic_df